In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import Select

# Carpeta de donde se guardará el archivo con los datos
download_folder = "C:\\Users\\angel\\Desktop\\MASTER" 

options = webdriver.ChromeOptions()  # Opciones de navegación con selenium

# Configuraciones de descarga en modo headless
# options.add_argument("--headless")  # Comentar esta línea si se desea ver la ejecución en modo no headless
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")

prefs = {
    "download.default_directory": download_folder,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    "safebrowsing.disable_download_protection": True,
    "profile.default_content_setting_values.automatic_downloads": 1, # Permitir múltiples descargas
}

options.add_experimental_option("prefs", prefs)

# Para asegurarse de que no se bloqueen las descargas en modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
# Configuración del driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Acceder a la página web
driver.get('https://www3.gobiernodecanarias.org/agricultura/agroclimatica/#/extraccion-datos')  

# Rellenar el formulario
# Paso 1: Escribe 'Santa Cruz de Tenerife' y selecciona de la lista de autocompletado
input_lugar = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[placeholder="Seleccione una de las opciones disponibles"]'))
)
input_lugar.send_keys('Santa Cruz de Tenerife')

# Espera a que la sugerencia sea visible y clickeable y luego selecciona 'Santa Cruz de Tenerife'
sugerencia_lugar = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//li[contains(., 'Santa Cruz de Tenerife')]"))
)
sugerencia_lugar.click()

# Paso 2: Escribe 'TF164 - La Laguna' y selecciona de la lista de autocompletado
input_codigo = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[placeholder="Seleccione una de las opciones disponibles"].undefined'))
)
input_codigo.send_keys('TF164 - La Laguna')

# Espera a que la sugerencia sea visible y clickeable y luego selecciona 'TF164 - La Laguna'
sugerencia_codigo = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//li[contains(., 'TF164 - La Laguna')]"))
)
sugerencia_codigo.click()

# Paso 3: Escribe 'Horario' en el campo correspondiente
# Encuentra todos los elementos de entrada con el mismo placeholder y selecciona el tercero (índice 2)
inputs = driver.find_elements(By.CSS_SELECTOR, 'input[placeholder="Seleccione una de las opciones disponibles"]')
input_horario = inputs[2]  # Esto asume que es el tercer campo de entrada
input_horario.click()  # Hacemos clic para activar el autocompletado

# Ahora escribe 'Horario' y espera a que la opción de autocompletado esté visible
input_horario.send_keys('Horario')

# Espera a que la opción 'Horario' en la lista de autocompletado sea visible y luego haz clic en ella
opcion_horario = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//li[contains(., 'Horario')]"))
)
opcion_horario.click()

def descargar_datos(fecha_inicio, fecha_fin):
    # Asegura que la fecha está en el formato correcto y termina con "00:00"
    fecha_inicio_str = fecha_inicio.strftime('%d/%m/%Y') + " 00:00"
    fecha_fin_str = fecha_fin.strftime('%d/%m/%Y') + " 00:00"

    # Encuentra el campo de la fecha 'Desde', selecciona todo el texto y borra cualquier valor existente
    input_fecha_desde = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'fechaDesde'))
    )
    ActionChains(driver) \
        .click(input_fecha_desde) \
        .key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL) \
        .send_keys(Keys.BACK_SPACE) \
        .send_keys(fecha_inicio_str) \
        .perform()

    # Espera un poco antes de pasar al siguiente campo
    time.sleep(0.5)

    # Encuentra el campo de la fecha 'Hasta', selecciona todo el texto y borra cualquier valor existente
    input_fecha_hasta = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'fechaHasta'))
    )
    ActionChains(driver) \
        .click(input_fecha_hasta) \
        .key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL) \
        .send_keys(Keys.BACK_SPACE) \
        .send_keys(fecha_fin_str) \
        .send_keys(Keys.ENTER) \
        .perform()
    
    time.sleep(2)
    
    # Click en 'Tablas de datos'
    btn_tablas_datos = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Tablas de datos')]"))
    )
    btn_tablas_datos.click()

    # Descargar datos
    btn_descargar = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn.btn-cardme-action.btn-downloader.table-downloader'))
    )
    btn_descargar.click()

    # Espera a que el elemento select esté presente y sea interactuable
    select_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'tipo_usuario'))
    )

    # Crea una instancia de Select pasando el elemento select encontrado
    select_objeto = Select(select_element)

    # Selecciona la opción "Investigador" por su valor
    select_objeto.select_by_value("INVESTIGADOR")

    # Espera a que el segundo elemento select esté presente y sea interactuable
    select_element_energias = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'aplicacion'))
    )

    # Crea una instancia de Select pasando el segundo elemento select encontrado
    select_objeto_energias = Select(select_element_energias)

    # Selecciona la opción "Energías renovables" por su valor
    select_objeto_energias.select_by_value("ENERGIAS_RENOVABLES")

    # Espera hasta que el botón 'Enviar' esté habilitado (es decir, que el atributo 'disabled' ya no esté presente)
    btn_enviar = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn-primary')][contains(@class, 'disable-while-loading') and not(@disabled)]"))
    )

    # Hacer clic en el botón 'Enviar'
    btn_enviar.click()
    
# Define el rango total de fechas para la descarga
fecha_inicio_total = datetime(2024, 3, 5)
fecha_fin_total = datetime(2024, 4, 5)

fecha_actual = fecha_inicio_total
while fecha_actual < fecha_fin_total:
    # Calcula la fecha fin para el segmento actual, que es dos días después de la fecha de inicio
    fecha_fin_segmento = fecha_actual + timedelta(days=1)

    # No superar la fecha fin total
    if fecha_fin_segmento > fecha_fin_total:
        fecha_fin_segmento = fecha_fin_total

    print(f"Descargando datos desde {fecha_actual.strftime('%d/%m/%Y')} hasta {fecha_fin_segmento.strftime('%d/%m/%Y')}")
    descargar_datos(fecha_actual, fecha_fin_segmento)

    # Avanza al siguiente segmento
    fecha_actual += timedelta(days=1)
    
    # Asegúrate de esperar un tiempo adecuado entre descargas para no sobrecargar el servidor o tu conexión
    time.sleep(10)


Descargando datos desde 05/03/2024 hasta 06/03/2024
Descargando datos desde 06/03/2024 hasta 07/03/2024
Descargando datos desde 07/03/2024 hasta 08/03/2024


KeyboardInterrupt: 